In [10]:
import pandas as pd
import numpy as np

In [20]:
all_meta = pd.read_pickle("all_meta.pkl", compression='bz2')

label_dict = {'muse':0, 'somaticsniper':1, 'either':2, np.nan:4}

all_meta['label'] = all_meta['label'].map(label_dict)

In [21]:
bucket = '<meta_data>'
prefix = 'sagemaker/gdc_meta'
 
# Define IAM role
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [22]:

rand_split = np.random.rand(len(all_meta))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

data_train = all_meta[train_list]
data_val = all_meta[val_list]
data_test = all_meta[test_list]


In [29]:
train_labels = np.array(data_train['label'])
val_labels = np.array(data_val['label'])
test_labels = np.array(data_test['label'])



In [31]:
train_features = np.array(data_train.drop(columns = ['label']))
val_features = np.array(data_val.drop(columns = ['label']))
test_features = np.array(data_test.drop(columns = ['label']))

In [34]:
import sagemaker
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

multiclass_estimator = sagemaker.LinearLearner(role=role,
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='multiclass_classifier',
                                               num_classes=4)

In [35]:
train_records = multiclass_estimator.record_set(train_features, train_labels, channel='train')
val_records = multiclass_estimator.record_set(val_features, val_labels, channel='validation')
test_records = multiclass_estimator.record_set(test_features, test_labels, channel='test')
multiclass_estimator.fit([train_records, val_records, test_records])

2019-07-02 21:50:48 Starting - Starting the training job...
2019-07-02 21:50:56 Starting - Launching requested ML instances.........
2019-07-02 21:52:34 Starting - Preparing the instances for training......
2019-07-02 21:53:36 Downloading - Downloading input data...
2019-07-02 21:54:19 Training - Training image download completed. Training in progress..
Docker entrypoint called with argument(s): train
[07/02/2019 21:54:21 INFO 139993019283264] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr

ValueError: Error for Training job linear-learner-2019-07-02-21-50-48-148: Failed Reason: ClientError: Unable to read data channel 'train'. Found missing (NaN) values. Please remove any missing (NaN) values in the input data. (caused by MXNetError)

Caused by: [21:54:21] /opt/brazil-pkg-cache/packages/AIAlgorithmsCppLibs/AIAlgorithmsCppLibs-2.0.1504.0/AL2012/generic-flavor/src/src/aialgs/io/iterator_base.cpp:103: (Input Error) (NaN) NaN value encountered in the dataset.

Stack trace returned 10 entries:
[bt] (0) /opt/amazon/lib/libaialgs.so(+0xb1f0) [0x7f52a87491f0]
[bt] (1) /opt/amazon/lib/libaialgs.so(+0xb54a) [0x7f52a874954a]
[bt] (2) /opt/amazon/lib/libaialgs.so(aialgs::iterator_base::Next()+0x3b0) [0x7f52a8752340]
[bt] (3) /opt/amazon/lib/libmxnet.so(MXDataIterNext+0x21) [0x7f5297228131]
[bt] (4) /opt/amazon/python2.7/lib/python2.7/lib-dynload/_ctypes.so(ffi_call_unix64+0x4c) [0x7f52a8ca4858]
[bt] (5) /opt/amazon/python2.7/lib/python2.7/lib-dynload/_ctypes.so(ffi_call+0x15f) [0x7f